<h1><center>Model for other Notebooks</center></h1>

# Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
from utils import *
import utils.transformers.transformers as tr
import utils.transformers.sk4pandas as s4p
import utils.transformers.knimp as knp

import os
import pickle
import urllib.request
import warnings

import git
from IPython.core.magic import register_cell_magic
import mlflow
import pandas as pd
from sklearn import set_config
from sklearn.base import clone
from sklearn.compose import make_column_selector
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, loguniform

warnings.filterwarnings('ignore')
set_config(display='text')

git_repo = git.Repo(os.getcwd())

@register_cell_magic
def run_and_save(line, cell):
    'Run and save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, line, 'exec')
    exec(code, globals())

# Init Datasets

In [ ]:
raw_main_df = pd.read_csv('data/train.csv')
raw_sub_df = pd.read_csv('data/test.csv')
raw_train_df, raw_test_df = train_test_split(raw_main_df, test_size=100, random_state=42)

X_all, y_all = raw_main_df.drop('Survived', axis=1), raw_main_df['Survived']
X_train, y_train = raw_train_df.drop('Survived', axis=1), raw_train_df['Survived']
X_test, y_test = raw_test_df.drop('Survived', axis=1), raw_test_df['Survived']

raw_train_df.sample(5, random_state=42)

# Prepare Pipeline

In [ ]:
%%run_and_save blueprints/pipeline.py


clip_outliers = [(
    'Float',
    tr.ClipOutliers(std_band=3),
    make_column_selector(dtype_include=['float64'])
)]

fillna_mean_mfreq = [(
    'Mean',
    s4p.SimpleImputer(strategy='mean'),
    make_column_selector(dtype_include=['float64'])
),(
    'Most Frequent',
    s4p.SimpleImputer(strategy='most_frequent'),
    make_column_selector(dtype_include=['int64', 'object'])
)]

fillna_constant = [(
    'Cabin',
    s4p.SimpleImputer(strategy='constant', fill_value='UNKNOWN'),
    ['Cabin']
),(
    'Age',
    s4p.SimpleImputer(strategy='constant', fill_value=-9999.),
    ['Age']
)]


steps_main = [
    ('Set working columns', tr.SetupFeatures(cols_ignore=['PassengerId', 'Name', 'Ticket']) ),
    ('Format Cabins', tr.Cabin() ),
    ('Standardize', s4p.StandardScaler(cols_select=['Age', 'Fare']) ),
    ('Clip Outliers', tr.ClipOutliers(cols_select=['Age', 'Fare'], std_band=3)),
    ('Replace NaNs With Constants', s4p.ColumnTransformer(fillna_constant, remainder='passthrough') ),
    ('Prepare coltypes for imputation', tr.AsTypes() ),
    ('Impute NaNs left with KNN', knp.KNImp(min_frequency=20) ),
    ('OneHot Encoding', s4p.OneHotEncoder(cols_select=['Pclass', 'Sex', 'SibSp', 'Embarked']) ),
    ('Dummy Classifier', DummyClassifier())
]

pipe = Pipeline(steps_main)

# Grid Params

In [ ]:
%%run_and_save blueprints/grid_params.py


ml_params_distributions = {
#    'Clip Outliers__Float__std_band': [3.],
#    'LogisticRegression__C': uniform(0.1, 3.),
}

rs = RandomizedSearchCV (
    pipe,
    param_distributions = ml_params_distributions,
    n_iter=1,
    n_jobs=-1,
    refit=True,
    cv=CVSplitter(5, 80),
    return_train_score=True
)

# ML flow run

In [ ]:
mlflow.sklearn.autolog(max_tuning_runs=3)

with mlflow.start_run(run_name='Dummy Classifier - New NaN imput method') as run:
    mlflow.set_tags(get_lastcommit_infos(git_repo))
    mlflow.log_artifact('blueprints/pipeline.py', 'blueprints')
    mlflow.log_artifact('blueprints/grid_params.py', 'blueprints')
    pipe.fit(X_all, y_all)
    #rs.fit(X_all, y_all)

# Refit best model

In [ ]:
run_id = '2ee74536f11e48189703684f2d03f45d'
url_local_model = f'file:///Users/i538262/Desktop/Developments/PERSO/KAGGLE%20Titanic/mlruns/0/{run_id}/artifacts/model/model.pkl' # model
url_local_model = f'file:///Users/i538262/Desktop/Developments/PERSO/KAGGLE%20Titanic/mlruns/0/{run_id}/artifacts/best_estimator/model.pkl' # best estimator (from rs)

with urllib.request.urlopen(url_local_model) as model_file:
    best_model = pickle.load(model_file)
    
best_model_allfit = clone(best_model)
best_model_allfit.fit(X_all, y_all)

accuracy_score(y_all, best_model_allfit.predict(X_all))

# Submit models predictions

In [ ]:
branch_name = get_lastcommit_infos(git_repo)['Branch']
submit(f'submissions/{branch_name}/{run_id}.csv', raw_sub_df.PassengerId, best_model_allfit.predict(raw_sub_df))

# Model insights

# Code tests